In [1]:
import os,sys,inspect
from ipywidgets import interact, interactive, fixed, interact_manual, Button
from IPython.display import display, Markdown, Latex, Javascript, clear_output, HTML
import emoji
import time
from datetime import date
import ipywidgets as widgets
import ipython_blocking
import numpy as np
from tqdm import tqdm_notebook
import functools
import configparser
import webbrowser
import shutil

parentdir = os.path.abspath('../../../pymodules')

sys.path.insert(0,parentdir)
#from connect_device import connect
from connect_device_v2.vizualizate import vizulizated_finds_devices, advanced_search_device,DeviceBundle
from connect_device_v2.save_file import SaveFilesButton
from connect_device_v2.connect import connect_to_device
from connect_device_v2.find_device import FindDevice
from connect_device_v2.auxiliary_commands import check_error,disable_all_correctors,enable_ifbf_for_all_segment
import pandas as pd
from datetime import datetime
button = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )

display(Markdown('### 🦢 Проверка устойчивости к механическому воздействию'))

### 🦢 Проверка устойчивости к механическому воздействию

In [2]:
from connect_device_v2.find_device import FindDevice
box = FindDevice()
display(box)

Выбранная конфигурация
filter_if:10 k
point_value:200
athenuator_value:10
cap_value:6554


<IPython.core.display.Javascript object>

In [11]:
button = box.select_device_continue_button

In [12]:
get_ipython().run_line_magic('blockrun', 'button')

In [13]:
button1 = box.select_config_continue_button

In [14]:
get_ipython().run_line_magic('blockrun', 'button1')

In [15]:
inst = box.selected_device

In [16]:
result_config =  box.config

In [17]:
def HZ_convert(pref, hertz):
    try:
        hertz = int(hertz)
    except ValueError:
        return -1
    if pref == "k":
        return hertz * 1000
    if pref == "M":
        return hertz * 1000000
    if pref == "G":
        return hertz * 1000000000
    return -1

def one_scan(inst):
    inst.write('TRIG:SOUR BUS')  # устанавливаем источник триггера на шину
    inst.write(f"INIT:CONT OFF")  # включаем повторный режим запуска триггера
    # переходим в состояние ожидания триггера
    inst.write('INIT')
    inst.write('TRIG:SING')  # выполняем россчерк
    inst.query('*OPC?') 

In [18]:
value_dict = {}

In [19]:
out = widgets.HTML()


In [20]:
if_split = result_config['DEVICE']['filter_IF'].split(" ")
if_value = f'{HZ_convert(if_split[1], int(if_split[0]))}'
at_value = result_config['DEVICE']['athenuator_value']
poin_value = result_config['DEVICE']['point_value']
cap_value = result_config['DEVICE']['cap_value']   

In [21]:
def param(if_value,poin_value,at_value,cap_value):
    inst.write(f'SYST:PRES')
    inst.write(f'SENS:SWE:TYPE LIN')
    
    #Включаем аттюнатор 
    inst.write(f'SERV:RFCTL:POW:STAT 1')
    inst.write(f'SERV:RFCTL:POW:ATT {at_value}')

    #Фильтр ПЧ
    inst.write(f'SENS:BWID {if_value}')
    time.sleep(1) 

    #Код ЦАП
    inst.write(f'SERV:RFCTL:POW:DAC {cap_value}')
    
    #количество точек.
    inst.write(f'SENS:SWE:POIN {poin_value}')


In [22]:
def setport(port):
    inst.write(f'CALC1:PAR:COUN 1')
    inst.write(f'CALC1:PAR1:DEF S{port}{port}')

In [23]:
def normalize(inst):
    inst.write(f'CALC:PAR1:SEL')
    inst.write(f'CALC:MATH:MEM')
    inst.write(f'CALC:MATH:FUNC DIV')
    

In [41]:

def gridbutton(count, value_dict):
    row = 0
    port_count = 0
    
    if count%6 == 0:
        row_1 = count//6
    else:
        row_1 = count//6+1
    c = 6
    grid = widgets.GridspecLayout(row_1 * 3, 6)
    for i in range(row_1):
        if i == row_1 - 1:
            c = count - c * i
        for j in range(c):
            port_count += 1
            g1 = widgets.Button(layout=widgets.Layout(width='160px', height='80px',border='solid'),description=f'До воздействия')
            g1.on_click(functools.partial(before_on_button_clicked, rs_=f'{port_count}'))

            g2 =  widgets.Button(layout=widgets.Layout(width='160px', height='80px',border='solid'),description=f'После воздействия')
            g2.on_click(functools.partial(after_on_button_clicked, value_dict=value_dict, rs_=f'{port_count}'))
#             Port_Button = widgets.Label(value = r'\(\color{Gold}  {''Портевич:' + str(port_count)  + '}\)',layout=widgets.Layout(display="flex",width='160px', height='80px',align_items='center', justify_content = 'center'))
            Port_Button = widgets.HTML(value=''' <style>
h4 {
  border: 3px solid;
  margin: 0;
  display: flex;
  transform:translateY(50px);
  justify-content: center;
  align-items:center;
  background-color:var(--jp-layout-color2);
}
</style>
<h4> Порт :''' + f' {port_count}</h4>')
#             Port_Button.style.button_color = 'lightgreen'
#             Port_Button.style.text_color = 'lightgreen'
            grid[row + 0, j] = Port_Button
            grid[row + 1, j] = g1
            grid[row + 2, j] = g2
        row += 3
    display(grid)


In [25]:
param(if_value,poin_value,at_value,cap_value)


date_inf = f'{date.today()}'
time_inf = inst.query(f'SYST:TIME?')

In [26]:
def marksearch(value_dict):
        
    trace = inst.query(f'CALC1:PAR1:DEF?')
    inst.write(f'CALC1:TRAC1:MARK1:STAT ON')
    inst.write(f'CALC1:TRAC1:MARK2:STAT ON')
    inst.write(f'CALC1:MARK1:FUNC:TYPE MAX')
    inst.write(f'CALC1:MARK2:FUNC:TYPE MIN')

    inst.write(f'CALC1:MARK1:FUNC:EXEC')
    inst.write(f'CALC1:MARK2:FUNC:EXEC')

    
    max_value = format(abs(float(inst.query(f'CALC1:MARK1:Y?').split(',')[0])),'.9f')
    min_value = format(abs(float(inst.query(f'CALC1:MARK2:Y?').split(',')[0])),'.9f')
    if max_value > min_value:
        value_dict.update({f'{trace}': max_value})
    else:
        value_dict.update({f'{trace}': min_value})
    return value_dict

In [27]:
table_dictionary = {}
def table_saver(table_dictionary, port_data, trace_value):
    table_dictionary.update({port_data:trace_value})

In [28]:
def tablehtml(value_dict, table_dictionary): 
    htmlstr = "<table border='1' ><tr><td>Порт</td><td>Отклонение</td><td>Максимум, дБ</td></tr><tr>"
    count = 0
    MAX = -1
    res_value_dict = {}
    res_value_dict.update(value_dict)
    for trace in res_value_dict.keys():
        if float(res_value_dict[f'{trace}']) > float(MAX):
            MAX = res_value_dict[f'{trace}']
    for trace in res_value_dict.keys():
        trace_value = res_value_dict[f'{trace}']
        cut = (len(trace)-1)//2 + 1
        port_data = trace[cut:]
#         port_data = trace
        if trace_value == MAX:
            htmlstr += f'<td>{port_data}</td><td class = bg-warning>{trace_value}</td>'
            table_saver(table_dictionary, port_data, trace_value)
        else:
            htmlstr += f'<td>{port_data}</td><td>{trace_value}</td>'
            table_saver(table_dictionary, port_data, trace_value)
        if count == 0:
            htmlstr += f'<td rowspan={len(res_value_dict.keys())+1}>{MAX}</td>'
            count += 1
        htmlstr += "</tr><tr>"
    htmlstr += '''</tr><style type="text/css">
td{
    padding: 2px;
    border-collapse: collapse;
    background-color:palegreen;
}
tr{
   padding: 2px;
   color: blue;
   border-collapse: collapse;
   font-family:verdana;
}
tr{
   font-size:20px;
   color:black;
   border-collapse: collapse;
   text-align:center;
}
</style>'''
    return htmlstr



In [29]:
def teblehader():
    idn_inf = inst.query(f'*IDN?').split(', ')
    htmlstr = "<table border='1' ><tr><td> Дата запуска </td><td> Время запуска </td><td>  device </td><td> model </td><td> serial </td><td>  version </td></tr><tr>"
    htmlstr += f'<td>{date_inf}</td><td>{time_inf}</td><td>{idn_inf[0]}</td><td>{idn_inf[1]}</td><td>{idn_inf[2]}</td><td>{idn_inf[3]}</td>'
    return htmlstr

In [30]:
def before_on_button_clicked(b, rs_="some_default_string"):
    setport(rs_)
    one_scan(inst)
    normalize(inst)
    one_scan(inst)
    b.button_style = 'success'

In [47]:
def after_on_button_clicked(b, value_dict, rs_="some_default_string"):
    setport(rs_)
    one_scan(inst)
    marksearch(value_dict)
    sort_on_button_clicked(b, table_dictionary)
#     out.value = teblehader() + tablehtml(value_dict, table_dictionary)
    
    b.button_style = 'success'
    global save_button
    save_button.file_data = teblehader() + tablehtml(value_dict, table_dictionary)


In [32]:
def dict_keys_to_type(a, Type):
    new_a = {}
    for key in a:
        if Type == str:
            new_a.update({' ' + (Type(key) * 2):a[key]})
        else:
            new_a.update({Type(key):a[key]})
    return(new_a)

In [45]:
def sort_on_button_clicked(b, table_dictionary):
    table_dictionary = dict_keys_to_type(table_dictionary, int)
    table_dictionary = dict(sorted(table_dictionary.items()))
    table_dictionary = dict_keys_to_type(table_dictionary, str)
    out.value = teblehader() + tablehtml(table_dictionary, table_dictionary)
    
    
#     g2 =  widgets.Button(layout=widgets.Layout(width='160px', height='80px',border='solid'),description=f'После воздействия')
#     g2.on_click(functools.partial(after_on_button_clicked, value_dict=value_dict, rs_=f'{port_count}'))

In [48]:

gridbutton(int(inst.query(f'SERV:PORT:COUN?')), value_dict)
display(out)
# sort_button=
save_button = SaveFilesButton(out.value)
sort_button = widgets.Button(layout=widgets.Layout(width='160px', height='80px',border='solid'),description=f'Сортировка')
sort_button.on_click(functools.partial(sort_on_button_clicked, table_dictionary = table_dictionary))
display(sort_button)
display(save_button)



GridspecLayout(children=(HTML(value=' <style>\nh4 {\n  border: 3px solid;\n  margin: 0;\n  display: flex;\n  t…

HTML(value='<table border=\'1\' ><tr><td> Дата запуска </td><td> Время запуска </td><td>  device </td><td> mod…

Button(description='Сортировка', layout=Layout(border='solid', height='80px', width='160px'), style=ButtonStyl…

SaveFilesButton(description='SaveFiles', icon='square-o', style=ButtonStyle(button_color='orange'))

{'2': '0.004574620', '3': '0.005115470', '4': '0.011542100', '1': '0.015300400', '5': '26.387000000', '6': '7.848320000', '14': '0.008123300', '13': '0.018119800', '10': '0.007643230', '11': '0.002770070', '16': '0.006847640', '15': '0.003120070', '9': '0.003751680', '8': '0.001538140'}
{'2': '0.004574620', '3': '0.005115470', '4': '0.011542100', '1': '0.015300400', '5': '26.387000000', '6': '7.848320000', '14': '0.008123300', '13': '0.018119800', '10': '0.007643230', '11': '0.002770070', '16': '0.006847640', '15': '0.003120070', '9': '0.003751680', '8': '0.001538140'}
{' 11': '0.015300400', ' 22': '0.004574620', ' 33': '0.005115470', ' 44': '0.011542100', ' 55': '26.387000000', ' 66': '7.848320000', ' 88': '0.001538140', ' 99': '0.003751680', ' 1010': '0.007643230', ' 1111': '0.002770070', ' 1313': '0.018119800', ' 1414': '0.008123300', ' 1515': '0.003120070', ' 1616': '0.006847640'}
{'2': '0.006113250', '3': '0.005115470', '4': '0.011542100', '1': '0.015300400', '5': '26.387000000', 